In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# ----- make nice figures -----
# Make 3d plots
from mpl_toolkits.mplot3d import Axes3D
import matplotlib as mpl
mpl.rcParams['figure.dpi']= 400

from cycler import cycler
COLORS = ['#F00D2C', '#242482', '#0071BE', '#4E8F00', '#553C67', '#DA5319']
default_cycler = cycler(color=COLORS)
plt.rc('axes', prop_cycle=default_cycler) 
# -----------------------------

# The Python requests library

Python has various http libraries that can be used to interact with REST APIs. Examples include:
* `http.client`
* `httplib2`
* `urllib3`

Another http library is the `requests` library, which is very easy to use. 
The documentation for this library can be found here:

http://docs.python-requests.org/en/master/

Below, we go over basic usage of this library, and how to use it to interact with the Materials Project API.

The first thing we do is import the library:

In [1]:
import requests

If you get an error above, you may need to intall the package on your computer, which you can do by running the cell below:

In [ ]:
!pip3 install requests --user

The next thing we do is to prepare the parts to the request. Depending on what method we're using and which resource we are accessing, we need to specify some of the following:

* HTTP Headers
* Resource URI
* Request Method
* Request Body

For example, below we're making a simple `GET` request to an API that requires an API key for authorization. This means we need to specify the HTTP header field to provide the key, as well as the URI for the resource:

In [2]:
# The resource we are requesting is material properties for Li-Fe-O material systems
uri = 'https://www.materialsproject.org/rest/v2/materials/Li-Fe-O/vasp'

# According to the Materials Project API documentation, we pass the API key in the X-API-KEY header field.
# We specify headers using a Python dictionary:
headers = {'X-API-KEY': 'v8ePsSkNCfTib7Kn'}

Once we define the components of the request, we make the request using the appropriate method of the `request` module. Available methods are:
* `requests.get`
* `requests.post`
* `requests.put`
* `requests.patch`
* `requests.delete`
* `requests.options`

How the API processes each of these requests methods depends on the specifics of that API but the whole idea of REST is that the API behaves in predictable ways, all according to REST conventions.

Below, we try to `GET` the resource described by the URI above using the `requests.get` function. Calling this function will initiate a request to the server, and wait for the server to send back a response, so it will take a little bit of time.

In [3]:
# Make the request, and get the response
r = requests.get(uri, headers=headers)

The actual form of the response depends on the API, and you should consult the documentation to see what is returned. We can se the raw text returned by the server, but often it's not that useful unless you know what you're looking at.

In [4]:
# Here's the raw form for the response
print(r.text)

{"response": [{"energy": -225.16232302, "energy_per_atom": -5.117325523181818, "volume": 417.9206613437442, "formation_energy_per_atom": -2.0107349745454544, "nsites": 44, "unit_cell_formula": {"Li": 24.0, "Fe": 4.0, "O": 16.0}, "pretty_formula": "Li6FeO4", "is_hubbard": true, "elements": ["Fe", "Li", "O"], "nelements": 3, "e_above_hull": 0.023674270681818044, "hubbards": {"Li": 0.0, "Fe": 5.3, "O": 0.0}, "is_compatible": true, "spacegroup": {"symprec": 0.1, "source": "spglib", "symbol": "P4_2/nmc", "number": 137, "point_group": "mmm", "crystal_system": "tetragonal", "hall": "P 4n 2n -1n"}, "task_ids": ["mp-812698", "mp-813302", "mp-771923", "mp-803893", "mp-756155", "mp-886632", "mp-1340463", "mp-1569033", "mp-1647309", "mp-1648271", "mp-1646144", "mp-1639402", "mp-1642132", "mp-1648509", "mp-1646270"], "band_gap": 2.2934, "density": 2.566593651994418, "icsd_id": null, "icsd_ids": [], "cif": "# generated using pymatgen\ndata_Li6FeO4\n_symmetry_space_group_name_H-M   'P 1'\n_cell_lengt

According to the API documentation, the server response is in JSON format, so we'll convert the JSON text into a Python dictionary

In [5]:
# We know this response is formated as json, so we'll convert it from a JSON object to a python dictionary
res = r.json()

# now res is a python dictionary
print(type(res))

<class 'dict'>


# Exploring the data using Python

Converting to a Python dictionary allows us to explore the response and associated data more in-depth. While you should always consult the documentation, we can do a small bit of exploration using some useful python functions.

Printing the full dictionary is as useful as printing the raw text, which is to say, not all that useful sometimes.

In [6]:
print(res)

{'response': [{'energy': -225.16232302, 'energy_per_atom': -5.117325523181818, 'volume': 417.9206613437442, 'formation_energy_per_atom': -2.0107349745454544, 'nsites': 44, 'unit_cell_formula': {'Li': 24.0, 'Fe': 4.0, 'O': 16.0}, 'pretty_formula': 'Li6FeO4', 'is_hubbard': True, 'elements': ['Fe', 'Li', 'O'], 'nelements': 3, 'e_above_hull': 0.023674270681818044, 'hubbards': {'Li': 0.0, 'Fe': 5.3, 'O': 0.0}, 'is_compatible': True, 'spacegroup': {'symprec': 0.1, 'source': 'spglib', 'symbol': 'P4_2/nmc', 'number': 137, 'point_group': 'mmm', 'crystal_system': 'tetragonal', 'hall': 'P 4n 2n -1n'}, 'task_ids': ['mp-812698', 'mp-813302', 'mp-771923', 'mp-803893', 'mp-756155', 'mp-886632', 'mp-1340463', 'mp-1569033', 'mp-1647309', 'mp-1648271', 'mp-1646144', 'mp-1639402', 'mp-1642132', 'mp-1648509', 'mp-1646270'], 'band_gap': 2.2934, 'density': 2.566593651994418, 'icsd_id': None, 'icsd_ids': [], 'cif': "# generated using pymatgen\ndata_Li6FeO4\n_symmetry_space_group_name_H-M   'P 1'\n_cell_lengt

One thing we can do is print all the field names of the dictionary to get a description of what is returned:

Recall a Python dictionary (like JSON) is just a set of key-value pairs

In [7]:
for key in res: 
    print(key)

response
valid_response
created_at
version
copyright


Here we see that the top level fields in the dictionary contain response meta information, and the actual response is contained in the `response` field. We can get this actual response by:

In [8]:
res = res['response']

We can then see what kind of response was returned using the `type` function:

In [9]:
type(res)

list

We see that the response is a list of things. Let's look at an example element (say the first one) in the list:

In [10]:
print(type(res[0]))

<class 'dict'>


In [ ]:
print(res[0])

It looks like each element of the list is a dictionary containing various material properties. The API documentation says this as well. Let's print out the properties that were returned:

In [11]:
for prop in res[0]:
    print(prop)

energy
energy_per_atom
volume
formation_energy_per_atom
nsites
unit_cell_formula
pretty_formula
is_hubbard
elements
nelements
e_above_hull
hubbards
is_compatible
spacegroup
task_ids
band_gap
density
icsd_id
icsd_ids
cif
total_magnetization
material_id
oxide_type
tags
elasticity
piezo
diel
full_formula


We can assume that each list element probably contains all of these properties. Let's print the `full_formula` for this first element: 

In [21]:
print(res[0]['unit_cell_formula'])

{'Li': 24.0, 'Fe': 4.0, 'O': 16.0}


Let's print out the formula for everything else in this list to see just what materials were returned in the response.

In [13]:
# Iterate through each element in the `res` list
for mat in res:
    print(mat['full_formula'])

Li24Fe4O16
Li5Fe7O12
Li2Fe5O10
Li2Fe2O4
Li5Fe5O10
Li16Fe16O32
Li10Fe8O16
Li18Fe4O16
Li2Fe4O6
Li10Fe6O16
Li6Fe4O12
Li16Fe4O12
Li1Fe2O4
Li2Fe4O8
Li2Fe2O6
Li4Fe4O8
Li12Fe4O16
Li16Fe2O12
Li16Fe4O16
Li3Fe4O8
Li2Fe6O8
Li6Fe4O12
Li3Fe5O12
Li3Fe7O12
Li12Fe4O12
Li4Fe3O8
Li11Fe4O12
Li1Fe5O8
Li20Fe4O16
Li12Fe4O16
Li3Fe3O8
Li2Fe4O6
Li20Fe4O16
Li20Fe4O16
Li2Fe16O24
Li1Fe1O2
Li20Fe4O16
Li6Fe4O12
Li12Fe12O36
Li32Fe8O36
Li8Fe4O12
Li8Fe40O64
Li16Fe10O20
Li8Fe7O15
Li4Fe8O16
Li3Fe3O8
Li8Fe4O12
Li12Fe20O32
Li6Fe4O12
Li6Fe6O16
Li8Fe14O24
Li2Fe6O12
Li9Fe2O8
Li1Fe5O8
Li6Fe7O15
Li16Fe10O20
Li18Fe12O36
Li3Fe1O6
Li3Fe5O10
Li6Fe5O12
Li6Fe1O6
Li6Fe7O15
Li5Fe4O8
Li6Fe10O24
Li1Fe3O4
Li4Fe4O12
Li35Fe8O32
Li7Fe5O12
Li12Fe4O12
Li6Fe6O16
Li4Fe4O8
Li4Fe4O12
Li30Fe8O32
Li8Fe7O15
Li1Fe3O6
Li6Fe6O16
Li12Fe4O12
Li6Fe4O12
Li2Fe16O24
Li2Fe2O6
Li2Fe4O8
Li8Fe4O8
Li15Fe2O12
Li4Fe6O12
Li10Fe10O24
Li6Fe10O18
Li4Fe6O12
Li3Fe2O6
Li8Fe16O32
Li3Fe2O6
Li1Fe1O3
Li3Fe4O8
Li8Fe4O8
Li8Fe16O32
Li7Fe5O12
Li8Fe4O8
Li2Fe4O8
Li3Fe1O4
Li18Fe4O16

Just a reminder, we got this specific list of materials because we asked for anything that contained Li O or Fe, when we sent the GET request to the url:

In [ ]:
print(uri)

We could print out other properties as well:

In [ ]:
# Iterate through each element in the `res` list
for mat in res:
    print(mat['full_formula'] + ": " + str(mat['band_gap']))

In [ ]:
# Iterate through each element in the `res` list
for mat in res:
    print(mat['full_formula'] +": " + str(mat['diel']))

We see that sometimes, some properties are not specified for all the results.

More more specifics about what is returned by Materials Project, consult the Materials Project wiki:

https://wiki.materialsproject.org/The_Materials_API#materials_.28calculated_materials_data.29

# Building a regression model based on materials project data

Let's try to build a regression model that maps composition to band gap. First, we'll need to encode the composition in a more useful form, other than the chemical formula. For example, we can describe each system:

$$ \text{Li}_x\text{Fe}_y\text{O}_z \mapsto (x, y, z).$$

Then we can try to model bandgap as a function of composition: $f(x, y, z)$ using a regression model.

So we'll need to convert the formula in the `full_formula` field into a triple of numbers $(x, y, z)$.

The full formula field be in the form of `Li#Fe#O#`, where # is a number. Also, we see that sometimes some
elements are not present in the formulas, which we'll interpret as a "0" in the composition vector. To extract
these numbers, we'll use **regular expressions** and the Python `re` library.

Regular expressions are used to search for and extract patterns from text by specifying a **search pattern string**. They can get complicated, so we won't go into them in depth here. For more information, look at this HOWTO:

https://docs.python.org/3/howto/regex.html#regex-howto

The search string we'll use to extract $(x, y, z)$ is:

```(?:Li(\d+))?(?:Fe(\d+))?(?:O(\d+))?```

The basic run-down of this expression is:
* Anything `(...)?` represents a pattern that may or may not occur in the string.
* The `(?:...)` represents a pattern that is searched for but not saved when extracting patterns from the string.
* The `(\d+)` is what we use to extract the composition for each. It says to save any number that appears in the pattern.

So: `(?:Li(\d+))?` represents a pattern `Li#`, which may or may not appear in the string. If it does, save the number.

In [14]:
import re
p = re.compile(r'(?:Li(\d+))?(?:Fe(\d+))?(?:O(\d+))?')

formula1 = "Li2Fe2O6"
m = p.match(formula1)
print(m.groups())

('2', '2', '6')


Note this matches if an element doesn't appear in the formula.

In [15]:
formula2 = "Fe2O6"
m = p.match(formula2)
print(m.groups())

(None, '2', '6')


Now we can build our data set $\left\{(x, y, z, b)\right\}$

In [18]:
!pip3 install numpy --user

     |████████████████████████████████| 8.0MB 5.2MB/s eta 0:00:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Created wheel for numpy: filename=numpy-1.20.0-cp38-cp38-macosx_10_14_6_arm64.whl size=9330672 sha256=d56f5a67fc9ca2e74e31d006e7f9e27b4eb1313961457828f3f6789d0878d83c
  Stored in directory: /Users/kreyes/Library/Caches/pip/wheels/87/1b/37/605372ce5ceabcb4543bb495b8b162da0b48b752ecdfdbc7e6
Successfully built numpy
You should consider upgrading via the 'pip install --upgrade pip' command.


In [22]:
import numpy as np
data = np.zeros([len(res), 4])

# Iterate through each element in the `res` list
for i, mat in enumerate(res):
    formula = mat['full_formula']
    m = p.match(formula)
    composition = (m.groups())
    
    # replace `None` with 0
    composition = [c if c is not None else 0 for c in composition]
    
    data[i,0:3] = composition
    data[i, 3] = mat['band_gap']

# I = (data[:, 3] > np.std(data[:, 3]))
# data = data[I, :]

In [25]:
print(data.shape)

(160, 4)


In [24]:
# Let's print out the first 10 data rows
print(data[0:10,:])

[[2.4000e+01 4.0000e+00 1.6000e+01 2.2934e+00]
 [5.0000e+00 7.0000e+00 1.2000e+01 1.2162e+00]
 [2.0000e+00 5.0000e+00 1.0000e+01 0.0000e+00]
 [2.0000e+00 2.0000e+00 4.0000e+00 0.0000e+00]
 [5.0000e+00 5.0000e+00 1.0000e+01 1.4360e+00]
 [1.6000e+01 1.6000e+01 3.2000e+01 1.8934e+00]
 [1.0000e+01 8.0000e+00 1.6000e+01 1.0737e+00]
 [1.8000e+01 4.0000e+00 1.6000e+01 2.4800e-02]
 [2.0000e+00 4.0000e+00 6.0000e+00 5.9950e-01]
 [1.0000e+01 6.0000e+00 1.6000e+01 0.0000e+00]]


Now we'll try to train a regression model to try to predict bandgap. We'll randomly permute the rows in case the API returned the results in some causal way.

In [26]:
np.random.shuffle(data)
print(data[0:10,:])

[[ 8.      5.     10.      0.    ]
 [24.      4.     16.      2.2934]
 [ 1.      3.      4.      0.7977]
 [ 3.      4.      8.      0.7019]
 [ 7.      1.      6.      0.3228]
 [ 1.      2.      4.      0.3124]
 [ 1.      1.      2.      1.7063]
 [20.      4.     16.      2.5866]
 [11.      4.     12.      0.    ]
 [32.      8.     36.      0.    ]]


In [27]:
# We'll split into testing and training set
num_training = int(0.8*len(data))
print(num_training)

X_train = data[0:num_training, 0:3]
y_train = data[0:num_training, 3]

X_test = data[num_training:, 0:3]
y_test = data[num_training:, 3]

128


In [28]:
# Normalize
mu_X = np.mean(X_train, axis = 0)
sig_X = np.std(X_train, axis = 0)
mu_y = np.mean(y_train)
sig_y = np.std(y_train)

# z-normalize X and y values
X_train_norm = (X_train - mu_X)/sig_X
y_train_norm = (y_train - mu_y)/sig_y

X_test_norm = (X_test - mu_X)/sig_X
y_test_norm = (y_test - mu_y)/sig_y

In [31]:
!pip3 install sklearn --user

     |████████████████████████████████| 7.4MB 3.6MB/s eta 0:00:01
  Installing build dependencies ... error
  ERROR: Command errored out with exit status 1:
   command: /Applications/Xcode.app/Contents/Developer/usr/bin/python3 /Applications/Xcode.app/Contents/Developer/Library/Frameworks/Python3.framework/Versions/3.8/lib/python3.8/site-packages/pip install --ignore-installed --no-user --prefix /private/var/folders/kw/jbfydz2n5gd5d1ytwm_hgnlw0000gn/T/pip-build-env-qx1w8kwp/overlay --no-warn-script-location --no-binary :none: --only-binary :none: -i https://pypi.org/simple -- setuptools wheel 'Cython>=0.28.5' 'numpy==1.13.3; python_version=='"'"'3.6'"'"' and platform_machine!='"'"'aarch64'"'"' and platform_system!='"'"'AIX'"'"' and platform_python_implementation=='"'"'CPython'"'"'' 'numpy==1.14.0; python_version=='"'"'3.6'"'"' and platform_machine!='"'"'aarch64'"'"' and platform_system!='"'"'AIX'"'"' and platform_python_implementation!='"'"'CPython'"'"'' 'numpy==1.16.0; python_version=

      compile options: '-Inumpy/core/src/common -Inumpy/core/src -Inumpy/core -Inumpy/core/src/npymath -Inumpy/core/src/multiarray -Inumpy/core/src/umath -Inumpy/core/src/npysort -I/Applications/Xcode.app/Contents/Developer/Library/Frameworks/Python3.framework/Versions/3.8/include/python3.8 -c'
      clang: _configtest.c
      removing: _configtest.c _configtest.o _configtest.o.d
      C compiler: clang -Wno-unused-result -Wsign-compare -Wunreachable-code -fno-common -dynamic -DNDEBUG -g -fwrapv -O3 -Wall -iwithsysroot/System/Library/Frameworks/System.framework/PrivateHeaders -iwithsysroot/Applications/Xcode.app/Contents/Developer/Library/Frameworks/Python3.framework/Versions/3.8/Headers -arch arm64 -arch x86_64
  
      compile options: '-Inumpy/core/src/common -Inumpy/core/src -Inumpy/core -Inumpy/core/src/npymath -Inumpy/core/src/multiarray -Inumpy/core/src/umath -Inumpy/core/src/npysort -I/Applications/Xcode.app/Contents/Developer/Library/Frameworks/Python3.framework/Versions/3.8/i

              npy_intp k;
              ^~~~~~~~~~~
      numpy/core/src/npysort/selection.c.src:326:14: note: silence by adding parentheses to mark code as explicitly dead
          else if (0 && kth == num - 1) {
                   ^
                   /* DISABLES CODE */ ( )
      numpy/core/src/npysort/selection.c.src:328:9: warning: code will never be executed [-Wunreachable-code]
              npy_intp k;
              ^~~~~~~~~~~
      numpy/core/src/npysort/selection.c.src:326:14: note: silence by adding parentheses to mark code as explicitly dead
          else if (0 && kth == num - 1) {
                   ^
                   /* DISABLES CODE */ ( )
      numpy/core/src/npysort/selection.c.src:328:9: warning: code will never be executed [-Wunreachable-code]
              npy_intp k;
              ^~~~~~~~~~~
      numpy/core/src/npysort/selection.c.src:326:14: note: silence by adding parentheses to mark code as explicitly dead
          else if (0 && kth == num - 1) {
       

ERROR: Command errored out with exit status 1: /Applications/Xcode.app/Contents/Developer/usr/bin/python3 /Applications/Xcode.app/Contents/Developer/Library/Frameworks/Python3.framework/Versions/3.8/lib/python3.8/site-packages/pip install --ignore-installed --no-user --prefix /private/var/folders/kw/jbfydz2n5gd5d1ytwm_hgnlw0000gn/T/pip-build-env-qx1w8kwp/overlay --no-warn-script-location --no-binary :none: --only-binary :none: -i https://pypi.org/simple -- setuptools wheel 'Cython>=0.28.5' 'numpy==1.13.3; python_version=='"'"'3.6'"'"' and platform_machine!='"'"'aarch64'"'"' and platform_system!='"'"'AIX'"'"' and platform_python_implementation=='"'"'CPython'"'"'' 'numpy==1.14.0; python_version=='"'"'3.6'"'"' and platform_machine!='"'"'aarch64'"'"' and platform_system!='"'"'AIX'"'"' and platform_python_implementation!='"'"'CPython'"'"'' 'numpy==1.16.0; python_version=='"'"'3.6'"'"' and platform_machine!='"'"'aarch64'"'"' and platform_system=='"'"'AIX'"'"'' 'numpy==1.16.0; python_version=

In [29]:
# Let's use kernel ridge regression with cross-validated regularization parameter alpha
from sklearn import linear_model

model = linear_model.RidgeCV(alphas = np.logspace(-5, 2, 20), cv = 5, fit_intercept=False)
model.fit(X_train_norm, y_train_norm)

ModuleNotFoundError: No module named 'sklearn'

In [ ]:
# Now let's predict and plot the results
y_model_norm = model.predict(X_test_norm)

# un-normalize to get units back
y_model = y_model_norm * sig_y  + mu_y

# plot
plt.scatter(y_model, y_test, c=COLORS[1])

# Plot the line y  = x
x_plot = [np.min(y_model), np.max(y_model)]
plt.plot(x_plot, x_plot, '--')

plt.grid()
plt.xlabel('Predicted bandgap (eV)')
plt.ylabel('True bandgap (eV)')

# Print some metrics
print("MSE:" + str(np.mean((y_model - y_test)**2)))

The fit isn't that great. Clearly, there is more going on. We could try to use additional properties, or may try to fit a subset of data. For example, let's look at the distribution band-gap values.

In [ ]:
# Let's plot the histogram for band gap values
plt.hist(data[:, 3], bins=20, rwidth=0.95)
plt.grid(axis='y')
plt.xlabel('True Bandgap value (eV)')
plt.ylabel('Frequency')

Lets try to predict semiconductor behavior by restricting data to those with a bandgap between e.g. 0.5 and 3 eV.

In [ ]:
# Pick out the indices of the rows where bandgap is greater than 0.5 and less than 3
I = (data[:, 3] > 0.5) & (data[:, 3] < 3)

# Extract those rows
data_semi = data[I, :]

Now repeat to fit a model.

In [ ]:
# We'll split into testing and training set
num_training = int(0.8*len(data_semi))

X_train = data_semi[0:num_training, 0:3]
y_train = data_semi[0:num_training, 3]
X_test = data_semi[num_training:, 0:3]
y_test = data_semi[num_training:, 3]

# Normalize
mu_X = np.mean(X_train, axis = 0)
sig_X = np.std(X_train, axis = 0)
mu_y = np.mean(y_train)
sig_y = np.std(y_train)
X_train_norm = (X_train - mu_X)/sig_X
y_train_norm = (y_train - mu_y)/sig_y
X_test_norm = (X_test - mu_X)/sig_X
y_test_norm = (y_test - mu_y)/sig_y

# fit
model = linear_model.RidgeCV(alphas = np.logspace(-5, 2, 20), cv = 5, fit_intercept=False)
model.fit(X_train_norm, y_train_norm)

# Now let's predict and plot the results
y_model_norm = model.predict(X_test_norm)

# un-normalize
y_model = y_model_norm *sig_y  + mu_y

# plot
plt.scatter(y_model, y_test, c=COLORS[1])

# Plot the line y  = x
x_plot = [np.min(y_model), np.max(y_model)]
plt.plot(x_plot, x_plot, '--')

plt.grid()
plt.xlabel('Predicted bandgap (eV)')
plt.ylabel('True bandgap (eV)')

# Print some metrics
print("MSE:" + str(np.mean((y_model - y_test)**2)))